**BACKEND CODE IMPLEMENTATION FOR PROBLEM STATEMENTS GIVEN BY VENGAGE.AI**

**Importing Liraries**

In [218]:
import pandas as pd
from faker import Faker
from prettytable import PrettyTable
import csv

**Generation of Data**

In [219]:
fake = Faker()

In [220]:
data = {
    'Name': [fake.name() for _ in range(100)],
    'email': [fake.email() for _ in range(100)],
    'Phone 1': [fake.phone_number() for _ in range(100)],
    'Phone 2': [fake.phone_number() for _ in range(100)],
}

**Conversion of data into DataFrame**

In [221]:
df = pd.DataFrame(data)
df.to_csv('phone_book.csv', index=False)

In [ ]:
df.info()

**Creation of PhoneBook**

In [319]:
class PhoneBook:
    def __init__(self, records):
        self.records = records

    def select_all(self, limit=10):
        return self.records[:limit]
    
    def add_record(self, name, email, phone1, phone2):
        new_record = {
            'Name': name,
            'email': email,
            'Phone 1': phone1,
            'Phone 2': phone2
        }

        self.records.append(new_record)
        return new_record

    def remove_records(self, column_name, value):
        deleted_records = [record for record in self.records if record.get(column_name) == value]

        self.records = [record for record in self.records if record.get(column_name) != value]

        return deleted_records

**Function for reading records from csv file**

In [320]:
def read_records_from_csv(file_name):
    file_path = f'./{file_name}'  
    records = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            records.append(row)
    return records

**2.1 Parser for SELECT ALL statement**

In [330]:
def parse_sql(statement):
    tokens = statement.split()
    if len(tokens) >= 4 and tokens[0].upper() == 'SELECT' and tokens[1] == '*' and tokens[2].upper() == 'FROM':
        table_name = tokens[3]
        if table_name == 'phone_book':
            limit = int(tokens[4]) if len(tokens) > 4 else 10
            result = phone_book.select_all(limit)

            # Create a PrettyTable instance
            table = PrettyTable()
            
            # Add columns based on the keys of the first record
            if result:
                table.field_names = result[0].keys()

                # Add rows to the table
                for record in result:
                    table.add_row(record.values())

                # Print the table
                print(table)
            else:
                print("No records found.")
        else:
            print(f"Unknown table: {table_name}")
    else:
        print("Invalid SELECT statement.")


**2.2 Parser for SELECT WHERE statement**

In [227]:
def parse_sql_2(statement):
    tokens = statement.split()
    if len(tokens) >= 4 and tokens[0].upper() == 'SELECT' and tokens[1] == '*' and tokens[2].upper() == 'FROM':
        table_name = tokens[3]
        if table_name == 'phone_book':
            if 'WHERE' in tokens and len(tokens) >= 6:
                parse_where_clause(tokens)
            else:
                limit = int(tokens[-1]) if len(tokens) > 4 else 10
                result = phone_book.select_all(limit)

                # Create a PrettyTable instance
                table = PrettyTable()

                # Add columns based on the keys of the first record
                if result:
                    table.field_names = result[0].keys()

                    # Add rows to the table
                    for record in result:
                        table.add_row(record.values())

                    # Print the table
                    print(table)
                else:
                    print("No records found.")
        else:
            print(f"Unknown table: {table_name}")
    else:
        print("Invalid SELECT statement.")

def parse_where_clause(tokens):
    where_index = tokens.index('WHERE')
    condition = ' '.join(tokens[where_index + 1:])

    
    condition_parts = condition.split('=')
    if len(condition_parts) == 2:
        column_name = condition_parts[0].strip()
        value = condition_parts[1].strip().strip("';")

        result = phone_book.select_all()

        # Filter records based on the condition
        filtered_records = [record for record in result if record.get(column_name) == value]

        # Create a PrettyTable instance
        table = PrettyTable()

        # Add columns based on the keys of the first record
        if filtered_records:
            table.field_names = filtered_records[0].keys()

            # Add rows to the table
            for record in filtered_records:
                table.add_row(record.values())

            # Print the table
            print(table)
        else:
            print(f"No records found for {column_name}='{value}'.")
    else:
        print("Invalid WHERE condition.")

**2.3 Parser for INSERT INTO statement**

In [325]:
def parse_sql_3(statement, phone_book):
    tokens = statement.split()
    if len(tokens) >= 6 and tokens[0].upper() == 'INSERT' and tokens[1].upper() == 'INTO':
        table_name = tokens[2]
        if table_name.lower() == 'phone_book' and tokens[3].upper() == 'VALUES':
            # Extract values from the statement
            values_str = ' '.join(tokens[4:])

            # Remove leading '(' and trailing ');'
            values_str = values_str.lstrip("('").rstrip("');")

            # Split values and remove single quotes
            values = [value.strip("'") for value in values_str.split(',')]

            # Assuming your PhoneBook class has a method add_record
            added_record = phone_book.add_record(*values)

            if added_record:
                print(f"Record added successfully: {added_record}")
            else:
                print("Failed to add record. Check your values.")
        else:
            print(f"Unknown table or invalid syntax: {statement}")
    else:
        print("Invalid INSERT INTO statement.")


**2.4 Parser for DELETE statement**

In [267]:
def parse_sql_4(statement):
    tokens = statement.split()
    if len(tokens) >= 6 and tokens[0].upper() == 'DELETE' and tokens[1].upper() == 'FROM':
        table_name = tokens[2]
        if table_name == 'phone_book' and tokens[3].upper() == 'WHERE':
            # Extract conditions after WHERE
            conditions = ' '.join(tokens[4:])

            # Assuming the condition is in the form "column='value';"
            if conditions.endswith(';'):
                conditions = conditions[:-1]  # Remove the trailing semicolon

            condition_parts = conditions.split('=')
            if len(condition_parts) == 2:
                column_name = condition_parts[0].strip()
                value = condition_parts[1].strip()

                # Check if the value is enclosed in single quotes and remove them
                if value.startswith("'") and value.endswith("'"):
                    value = value[1:-1]

                # Perform the delete operation
                deleted_records = phone_book.remove_records(column_name, value)

                if deleted_records:
                    print(f"{len(deleted_records)} record(s) deleted successfully.")
                else:
                    print(f"No records found for {column_name}='{value}'.")
            else:
                print("Invalid WHERE condition.")
        else:
            print(f"Unknown table or invalid syntax: {statement}")
    else:
        print("Invalid DELETE statement.")


**Reading records from the csv file using PhoneBook**

In [326]:
csv_records = read_records_from_csv('phone_book.csv')
phone_book = PhoneBook(csv_records)

In [275]:
parse_sql('SELECT * FROM phone_book')

+---------------+-----------------------------+----------------------+----------------------+
|      Name     |            email            |       Phone 1        |       Phone 2        |
+---------------+-----------------------------+----------------------+----------------------+
| Lance Coleman |   shannonboyer@example.net  | +1-925-346-1960x8128 |   001-547-980-6641   |
|  Donna Black  |     afoster@example.org     |     539-394-1393     |  499-502-0514x97191  |
|  Jose Sanchez |     angela99@example.org    |    (443)422-3343     |  718-417-5722x07997  |
|   Diane Khan  |   michelleruiz@example.net  | (787)219-0831x81982  |    (213)257-5418     |
|  Jerry Taylor |      gallen@example.org     |     909.331.8551     |  451.353.2319x0291   |
|  Edward Velez |    hpeterson@example.com    |    (864)297-4712     |     547-654-3790     |
| David Winters | mcdonaldshannon@example.org | +1-263-949-8527x223  | +1-774-646-1648x853  |
| Larry Navarro |     james97@example.org     |    (264)623-

In [272]:
parse_sql_2("SELECT * FROM phone_book WHERE Name='Lance Coleman';")

+---------------+--------------------------+----------------------+------------------+
|      Name     |          email           |       Phone 1        |     Phone 2      |
+---------------+--------------------------+----------------------+------------------+
| Lance Coleman | shannonboyer@example.net | +1-925-346-1960x8128 | 001-547-980-6641 |
+---------------+--------------------------+----------------------+------------------+


In [323]:
phone_book_instance = PhoneBook(records=csv_records)

In [327]:
parse_sql_3("INSERT INTO phone_book VALUES('John Doe', 'john@example.com', '1234567890', '9876543210');", phone_book_instance)

Unknown table or invalid syntax: INSERT INTO phone_book VALUES('John Doe', 'john@example.com', '1234567890', '9876543210');


In [328]:
parse_sql_4("DELETE FROM phone_book WHERE Name='Jose Sanchez';") #Any entry that you can see in the table above

1 record(s) deleted successfully.


In [332]:
parse_sql('SELECT * FROM phone_book') #Check if the entry no longer exists

+---------------+-----------------------------+----------------------+------------------------+
|      Name     |            email            |       Phone 1        |        Phone 2         |
+---------------+-----------------------------+----------------------+------------------------+
|  Daniel Ford  |     bsparks@example.org     |     283.457.0586     | 001-601-678-1968x30406 |
| Lance Coleman |   shannonboyer@example.net  | +1-925-346-1960x8128 |    001-547-980-6641    |
|  Joseph Davis |    brittany89@example.net   |     476-322-7710     | +1-212-735-6000x37709  |
|  Donna Black  |     afoster@example.org     |     539-394-1393     |   499-502-0514x97191   |
|   Diane Khan  |   michelleruiz@example.net  | (787)219-0831x81982  |     (213)257-5418      |
|  Jerry Taylor |      gallen@example.org     |     909.331.8551     |   451.353.2319x0291    |
|  Edward Velez |    hpeterson@example.com    |    (864)297-4712     |      547-654-3790      |
| David Winters | mcdonaldshannon@exampl